In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

### Green GroupBy

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.createOrReplaceTempView('green')

In [5]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [6]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

### Yellow GroupBy

In [7]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [8]:
df_yellow.createOrReplaceTempView('yellow')

In [9]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [10]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

### Report Join

In [11]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [12]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [13]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [14]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

### Data Join

In [16]:
df_join = spark.read.parquet('data/report/revenue/total')

In [18]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  24|              87.6|                   3| 754.9500000000003|                   45|
|2020-01-01 00:00:00|  40|168.98000000000002|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  49|266.76000000000005|                  14|185.64999999999998|                   10|
|2020-01-01 00:00:00|  50|              null|                null| 4177.479999999998|                  183|
|2020-01-01 00:00:00|  68|              null|                null| 7825.070000000011|                  396|
|2020-01-01 00:00:00|  87|              null|                null| 2456.669999999999|                  112|
|2020-01-01 00:00:00|  92|  

In [19]:
df_zones = spark.read.parquet('zones/')

In [20]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [21]:
df_result.drop('LocationID', 'zone').show()

+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|2020-01-01 00:00:00|              87.6|                   3| 754.9500000000003|                   45|Manhattan| Yellow Zone|
|2020-01-01 00:00:00|168.98000000000002|                   8|             89.97|                    5| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|266.76000000000005|                  14|185.64999999999998|                   10| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|              null|                null| 4177.479999999998|                  183|Manhattan| Yellow Zone|
|2020-01-01 00:00:00|              null|                null| 7825.070000000011|                  396|Manhattan| Yello

In [22]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

### END